In [2]:
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import sdeint

#Model parameters
r = 1 #growth rate of the resource
K = 10 #carrying capacity
Vmax = 1 #biomass at which the grazing rate is half maximum
V0 = 6.6 #initial population V
c0 = 2.3 #initial grazing rate

#Multiscale entropy settings
nts = 5 #number of timscales
m = 2
tolerance = 'var' #tolerance has to be set manually in the code

#Run settings
run_nr = 'test' #name of the run
L = 1000 #length of datapoints considered
runs = 50
runtime = 330
resolution = 3301
distortion = np.diag([0.1, 0])
t = np.linspace(0, runtime, resolution)
y0 = np.array([V0, c0])

#Creating arrays for data
ts = np.zeros(shape=(runs, resolution))
sd = np.zeros(shape=(runs,resolution-L+1))
ac = np.zeros(shape=(runs,resolution-L+1))
mse = np.zeros(shape=(runs,resolution-L+1))
se = np.zeros(shape=(runs, resolution-L+1, nts))

#solving ODE model as a reference
def model(y, t, r, K, Vmax):
    V, c = y
    dydt = [r*V*(1-V/K)- (c*V**2)/(V**2+Vmax**2), 0.001]
    return dydt

sol_ode = odeint(model, y0, t, args=(r, K, Vmax))
c = sol_ode[:,1]

#Defining the SDE model
def f(y, t):
    V, c = y
    dydt = np.array([r*V*(1-V/K)- (c*V**2)/(V**2+Vmax**2), 0.001])
    return dydt
def g(y, t):
    return distortion

#Defining multiscale entropy (vectorized code from wikipedia)
def sampen(x, m, r):
    N = len(x)
    B = 0.0
    A = 0.0
    
    # Split time series and save all templates of length m
    xmi = np.array([x[i : i + m] for i in range(N - m)])
    xmj = np.array([x[i : i + m] for i in range(N - m + 1)])

    # Save all mat ches minus the self-match, compute B
    B = np.sum([np.sum(np.abs(xmii - xmj).max(axis=1) <= r) - 1 for xmii in xmi])
    
    # Similar for computing A
    m += 1
    xm = np.array([x[i : i + m] for i in range(N - m + 1)])

    A = np.sum([np.sum(np.abs(xmi - xm).max(axis=1) <= r) - 1 for xmi in xm])

    # Return SampEn
    return -np.log(A / B)

#Creating timescales for multiscale entropy
timescales = np.empty(nts)
for i in range(nts):
    timescales[i] = i + 1

#Running simulations of the SDE model
i = 0
while i < runs:
    sol_sde = sdeint.itoEuler(f, g, y0, t)
    ts[i,:] = sol_sde[:,0]
    #Deleting runs that went over tipping point
    if ts[i, resolution-1] < 4: 
        continue
    i = i + 1

#Computing standard deviation
for i in range(runs):
    for j in range(resolution-L+1):
        sd[i,j] = np.std(ts[i,j:j+L])

#Computing autocorrelation
for i in range(runs):
    for j in range(resolution-L+1):
        pearson = np.corrcoef(ts[i,j:j+L-1], ts[i,j+1:j+L])
        ac[i,j] = pearson[0,1]

#Computing sample entropy and multiscale entropy
for i in range(runs):
    #Coarse graining of time series
    for j in range(resolution-L+1):
        cts = np.zeros(shape=(nts, L), dtype=float)
        se_cts = np.empty(nts)
        for k in range(nts):
            for l in range(math.floor(L/(k+1))):
                cts[k,l] = np.sum(ts[i,j+l*(k+1):j+l*(k+1)+k+1])/(k+1)
            se_cts[k] = sampen(cts[k,:math.floor(L/(k+1))], 2, 0.2*np.std(ts[i,:]))
        se[i,j,:] = se_cts[:]
        mse[i,j] = np.trapz(se_cts, timescales)

#Saving data
np.save(run_nr+'_ts', ts)
np.save(run_nr+'_sd', sd)
np.save(run_nr+'_ac', ac)
np.save(run_nr+'_mse_'+tolerance, mse)
np.save(run_nr+'_se_'+tolerance, se)